In [10]:
import pandas as pd
import numpy as np
import datetime
from sim import generate_simulation
from model_cancellations import get_otb_res, predict_cancellations

pd.options.display.max_rows = 150
pd.options.display.max_columns = 150

In [11]:
DATE_FMT = "%Y-%m-%d"
h1_capacity = 187
h2_capacity = 226
AOD = "2016-07-01"
AOD_dt = pd.to_datetime(AOD)

h1_res = pd.read_pickle("pickle/h1_res.pick")
h2_res = pd.read_pickle("pickle/h2_res.pick")
h1_dbd = pd.read_pickle("pickle/h1_dbd.pick")
h2_dbd = pd.read_pickle("pickle/h2_dbd.pick")
# h1_sim = pd.read_pickle("pickle/h1_sim.pick")
# h2_sim = pd.read_pickle("pickle/h2_sim.pick")

In [15]:
df_sim

,Date,DOW,RoomsOTB,RevOTB,CxlForecast,Trn_RoomsOTB,Trn_RevOTB,Trn_CxlForecast,TrnP_RoomsOTB,TrnP_RevOTB,TrnP_CxlForecast,Cnt_RoomsOTB,Cnt_RevOTB,Cnt_CxlForecast,Grp_RoomsOTB,Grp_RevOTB,Grp_CxlForecast,Date,TM05_Date,TM15_Date,TM30_Date,WeekEndDate,WE,WD,STLY_Date,Occ,RevPAR,RemSupply,ADR_OTB,Trn_ADR_OTB,TrnP_ADR_OTB,Grp_ADR_OTB,Cnt_ADR_OTB,LYA_RoomsSold,LYA_ADR,LYA_RoomRev,LYA_RevPAR,LYA_NumCancels,SellingPrice,TM05_RoomsOTB,TM05_ADR,TM05_TRN_OTB,TM05_TRN_ADR,TM05_TRNP_OTB,TM05_TRNP_ADR,TM05_GRP_OTB,TM05_GRP_ADR,TM05_CNT_OTB,TM05_CNT_ADR,TM15_RoomsOTB,TM15_ADR,TM15_TRN_OTB,TM15_TRN_ADR,TM15_TRNP_OTB,TM15_TRNP_ADR,TM15_GRP_OTB,TM15_GRP_ADR,TM15_CNT_OTB,TM15_CNT_ADR,TM30_RoomsOTB,TM30_ADR,TM30_TRN_OTB,TM30_TRN_ADR,TM30_TRNP_OTB,TM30_TRNP_ADR,TM30_GRP_OTB,TM30_GRP_ADR,TM30_CNT_OTB,TM30_CNT_ADR
0,2016-08-02,Tue,183.0,31110.89,30.0,159.0,27829.05,28.0,14.0,2134.60,2.0,10.0,1147.24,0.0,0.0,0.0,0.0,2016-08-02,2016-07-28,2016-07-18,2016-07-03,2016-08-07,False,False,2015-08-04,0.98,166.37,34,170.00,175.03,152.47,0.0,114.72,182.0,151.57,27585.83,147.52,58.0,185.03,182.0,168.12,158.0,172.89,14.0,152.47,0.0,0.0,10.0,114.72,180.0,167.36,156.0,172.07,14.0,152.47,0.0,0.0,10.0,114.72,180.0,166.00,156.0,170.50,14.0,152.47,0.0,0.0,10.0,114.72
1,2016-08-03,Wed,182.0,31660.62,35.0,159.0,28337.88,31.0,14.0,2162.60,4.0,8.0,919.14,0.0,1.0,241.0,0.0,2016-08-03,2016-07-29,2016-07-19,2016-07-04,2016-08-07,False,False,2015-08-05,0.97,169.31,40,173.96,178.23,154.47,241.0,114.89,182.0,152.92,27831.73,148.83,59.0,185.03,173.0,172.87,151.0,177.65,14.0,154.47,0.0,0.0,8.0,114.89,176.0,172.51,154.0,177.14,14.0,154.47,0.0,0.0,8.0,114.89,179.0,170.49,157.0,174.75,14.0,154.47,0.0,0.0,8.0,114.89
2,2016-08-04,Thu,183.0,33326.17,35.0,153.0,28417.71,32.0,20.0,3731.12,3.0,10.0,1177.34,0.0,0.0,0.0,0.0,2016-08-04,2016-07-30,2016-07-20,2016-07-05,2016-08-07,False,False,2015-08-06,0.98,178.21,39,182.11,185.74,186.56,0.0,117.73,180.0,155.27,27947.92,149.45,57.0,185.03,177.0,180.40,149.0,184.72,18.0,179.47,0.0,0.0,10.0,117.73,177.0,179.67,149.0,183.85,18.0,179.47,0.0,0.0,10.0,117.73,173.0,177.01,145.0,180.79,18.0,179.47,0.0,0.0,10.0,117.73
3,2016-08-05,Fri,184.0,34607.31,30.0,150.0,28531.32,27.0,25.0,5050.29,3.0,9.0,1025.70,0.0,0.0,0.0,0.0,2016-08-05,2016-07-31,2016-07-21,2016-07-06,2016-08-07,True,False,2015-08-07,0.98,185.07,33,188.08,190.21,202.01,0.0,113.97,182.0,153.31,27901.89,149.21,59.0,185.03,176.0,187.25,142.0,189.30,25.0,202.01,0.0,0.0,9.0,113.97,176.0,185.35,144.0,187.83,23.0,197.81,0.0,0.0,9.0,113.97,173.0,181.01,143.0,184.21,21.0,187.93,0.0,0.0,9.0,113.97
4,2016-08-06,Sat,180.0,34211.53,28.0,148.0,28556.84,25.0,23.0,4628.99,3.0,9.0,1025.70,0.0,0.0,0.0,0.0,2016-08-06,2016-08-01,2016-07-22,2016-07-07,2016-08-07,True,False,2015-08-08,0.96,182.95,35,190.06,192.95,201.26,0.0,113.97,181.0,156.18,28269.01,151.17,63.0,185.03,173.0,188.63,141.0,191.33,23.0,201.26,0.0,0.0,9.0,113.97,173.0,186.56,143.0,189.66,21.0,196.59,0.0,0.0,9.0,113.97,171.0,183.22,143.0,187.27,19.0,185.54,0.0,0.0,9.0,113.97
5,2016-08-07,Sun,179.0,33400.44,22.0,147.0,27814.41,19.0,23.0,4560.33,3.0,9.0,1025.70,0.0,0.0,0.0,0.0,2016-08-07,2016-08-02,2016-07-23,2016-07-08,2016-08-07,False,False,2015-08-09,0.96,178.61,30,186.59,189.21,198.28,0.0,113.97,171.0,154.22,26371.82,141.03,62.0,185.03,170.0,186.15,138.0,188.84,23.0,198.28,0.0,0.0,9.0,113.97,168.0,184.68,136.0,187.06,23.0,198.28,0.0,0.0,9.0,113.97,167.0,181.16,135.0,182.72,23.0,198.28,0.0,0.0,9.0,113.97
6,2016-08-08,Mon,183.0,33828.45,38.0,148.0,27724.35,34.0,25.0,4961.63,4.0,10.0,1142.47,0.0,0.0,0.0,0.0,2016-08-08,2016-08-03,2016-07-24,2016-07-09,2016-08-14,False,False,2015-08-10,0.98,180.90,42,184.85,187.33,198.47,0.0,114.25,182.0,160.26,29166.44,155.97,61.0,193.74,180.0,183.64,145.0,185.87,25.0,198.47,0.0,0.0,10.0,114.25,178.0,182.50,144.0,184.32,24.0,200.02,0.0,0.0,10.0,114.25,177.0,179.67,145.0,181.95,22.0,194.39,0.0,0.0,10.0,114.25
7,2016-08-09,Tue,183.0,33625.94,42.0,142.0,26334.82,36.0,29.0,5787.58,5.0,12.0,1503.54,1.0,0.0,0.0,0.0,2016-08-0

In [16]:
df_sim = pd.read_csv("sims/h1_sim_2016-08-02.csv")
df_sim.rename(columns={"Unnamed: 0": 'Date'}, inplace=True, errors='ignore')

In [13]:
df_sim.head(3)

,Date,DOW,RoomsOTB,RevOTB,CxlForecast,Trn_RoomsOTB,Trn_RevOTB,Trn_CxlForecast,TrnP_RoomsOTB,TrnP_RevOTB,TrnP_CxlForecast,Cnt_RoomsOTB,Cnt_RevOTB,Cnt_CxlForecast,Grp_RoomsOTB,Grp_RevOTB,Grp_CxlForecast,Date,TM05_Date,TM15_Date,TM30_Date,WeekEndDate,WE,WD,STLY_Date,Occ,RevPAR,RemSupply,ADR_OTB,Trn_ADR_OTB,TrnP_ADR_OTB,Grp_ADR_OTB,Cnt_ADR_OTB,LYA_RoomsSold,LYA_ADR,LYA_RoomRev,LYA_RevPAR,LYA_NumCancels,SellingPrice,TM05_RoomsOTB,TM05_ADR,TM05_TRN_OTB,TM05_TRN_ADR,TM05_TRNP_OTB,TM05_TRNP_ADR,TM05_GRP_OTB,TM05_GRP_ADR,TM05_CNT_OTB,TM05_CNT_ADR,TM15_RoomsOTB,TM15_ADR,TM15_TRN_OTB,TM15_TRN_ADR,TM15_TRNP_OTB,TM15_TRNP_ADR,TM15_GRP_OTB,TM15_GRP_ADR,TM15_CNT_OTB,TM15_CNT_ADR,TM30_RoomsOTB,TM30_ADR,TM30_TRN_OTB,TM30_TRN_ADR,TM30_TRNP_OTB,TM30_TRNP_ADR,TM30_GRP_OTB,TM30_GRP_ADR,TM30_CNT_OTB,TM30_CNT_ADR
0,2016-08-02,Tue,183.0,31110.89,30.0,159.0,27829.05,28.0,14.0,2134.60,2.0,10.0,1147.24,0.0,0.0,0.0,0.0,2016-08-02,2016-07-28,2016-07-18,2016-07-03,2016-08-07,False,False,2015-08-04,0.98,166.37,34,170.00,175.03,152.47,0.0,114.72,182.0,151.57,27585.83,147.52,58.0,185.03,182.0,168.12,158.0,172.89,14.0,152.47,0.0,0.0,10.0,114.72,180.0,167.36,156.0,172.07,14.0,152.47,0.0,0.0,10.0,114.72,180.0,166.00,156.0,170.50,14.0,152.47,0.0,0.0,10.0,114.72
1,2016-08-03,Wed,182.0,31660.62,35.0,159.0,28337.88,31.0,14.0,2162.60,4.0,8.0,919.14,0.0,1.0,241.0,0.0,2016-08-03,2016-07-29,2016-07-19,2016-07-04,2016-08-07,False,False,2015-08-05,0.97,169.31,40,173.96,178.23,154.47,241.0,114.89,182.0,152.92,27831.73,148.83,59.0,185.03,173.0,172.87,151.0,177.65,14.0,154.47,0.0,0.0,8.0,114.89,176.0,172.51,154.0,177.14,14.0,154.47,0.0,0.0,8.0,114.89,179.0,170.49,157.0,174.75,14.0,154.47,0.0,0.0,8.0,114.89
2,2016-08-04,Thu,183.0,33326.17,35.0,153.0,28417.71,32.0,20.0,3731.12,3.0,10.0,1177.34,0.0,0.0,0.0,0.0,2016-08-04,2016-07-30,2016-07-20,2016-07-05,2016-08-07,False,False,2015-08-06,0.98,178.21,39,182.11,185.74,186.56,0.0,117.73,180.0,155.27,27947.92,149.45,57.0,185.03,177.0,180.40,149.0,184.72,18.0,179.47,0.0,0.0,10.0,117.73,177.0,179.67,149.0,183.85,18.0,179.47,0.0,0.0,10.0,117.73,173.0,177.01,145.0,180.79,18.0,179.47,0.0,0.0,10.0,117.73
